# Required Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import json
import re

In [ ]:
df = pd.read_csv('dataset.csv') #,index_col=False)

In [ ]:
! pip install datasets
! pip install transformers[torch]
! pip install tokenizers
! pip install evaluate
! pip install rouge_score
! pip install sentencepiece
! pip install huggingface_hub
!pip install timexy

In [ ]:
import nltk
import evaluate
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq,TFMT5ForConditionalGeneration, MT5Tokenizer, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
from datasets import Dataset, DatasetDict
import timexy
from timexy import Timexy
from datasets import Dataset, DatasetDict
from timexy import rule
from timexy.languages import en
import tensorflow
from tensorflow.keras.optimizers import Adam

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Optionally add config if varying from default values
config = {
    "kb_id_type": "timex3",  # possible values: 'timex3'(default), 'timestamp'
    "label": "timexy",       # default: 'timexy'
    "overwrite": False       # default: False
}
nlp.add_pipe("timexy", config=config, before="ner")

In [ ]:
nltk.download("punkt", quiet=True)

True

# Preprocessing

In [ ]:
def preprocess(batch):
    clean_batch = []
    for i in batch['Inputs']:
        for j in i.split('\n')[:-1]:
            x = re.split('~__(True|False)__',j)
            sent = x[0]
            if(eval(x[1])): # temporal
              sent = "(The following sentence has some temporal information.) " + j
            else:
              sent = "(The following sentence has no temporal information.) " + j
            clean_batch.append(sent)
    inputs = ["Generate : " + sentence for sentence in clean_batch]

    model_inputs = tokenizer(inputs,truncation=False,padding=True)
    labels = tokenizer(text_target=list(batch['Outputs']),truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def compute_metrics(eval_preds):
    preds,refs = eval_preds

    decode_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(refs, skip_special_tokens=True)

    pred_tags = nlp(decode_preds)
    actual_tags = [map(lambda x: tokenized_dataset.index(x),decoded_labels)]

    err = nltk.metrics.scores.log_likelihood(actual_tags,pred_tags)

    return err

In [ ]:
# train (1081)
c=0
for index, row in df.iterrows():
    if((row['Chunk'])<=1081):
        c+=1
train = df.iloc[:c,:]

# validate (rest)
test = df.iloc[c:,:]

# Training With T5

In [ ]:
# Run it the first time
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

chkpnt = './input/checkpoints/chkpnt1'
model.save_pretrained(chkpnt)
tokenizer.save_pretrained(chkpnt)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(chkpnt)
tokenizer = T5Tokenizer.from_pretrained(chkpnt)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
tokenized_dataset = preprocess(train)

In [ ]:
test_dataset = preprocess(test)

In [ ]:
# Training Arguments
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset,
   eval_dataset=test_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

In [ ]:
list(tokenized_dataset["input_ids"])

140284

# Training With T5 2

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = TFMT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=tokenizer.pad_token_id,
            pad_to_multiple_of=64,
            return_tensors="np",
        )

In [ ]:
train_dataset = Dataset.from_pandas(df).map(preprocess, batched=True, desc="Running tokenizer")

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
            train_dataset,
            collate_fn=data_collator,
            batch_size=8,
            shuffle=True,
        )

TypeError: Dataset argument should be a datasets.Dataset!

In [ ]:
print(train["Inputs"].iloc[1])
for i in train["Inputs"].iloc[1].split('\n'):
  x = re.split('~__(True|False)__',i)
  print(type(x[0]))

# Training with Auto T5

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenized_dataset = preprocess(train)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"{MODEL_NAME}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 1,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = False
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_dataset,
    eval_dataset = test_dataset,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()